In [23]:
from logicqubit.logic import *
from cmath import *
import numpy as np
import sympy as sp
import scipy
from scipy.optimize import *
import matplotlib.pyplot as plt

https://arxiv.org/abs/1304.3061

https://cpb-us-w2.wpmucdn.com/voices.uchicago.edu/dist/0/2327/files/2020/10/SimultaneousMeasurementVQE.pdf1

In [24]:
gates = Gates(1)

ID = gates.ID()
X = gates.X()
Y = gates.Y()
Z = gates.Z()

In [25]:
II = ID.kron(ID)

XX = X.kron(X)
XI = X.kron(ID)
IX = ID.kron(X)
XY = X.kron(Y)
XZ = X.kron(Z)

YY = Y.kron(Y)
YI = Y.kron(ID)
IY = ID.kron(Y)
YX = Y.kron(X)
YZ = Y.kron(Z)

ZZ = Z.kron(Z)
ZI = Z.kron(ID)
IZ = ID.kron(Z)
ZX = Z.kron(X)
ZY = Z.kron(Y)

sig_is = np.kron([1, 1], [1, -1])
sig_si = np.kron([1, -1], [1, 1])
ZZ.get()

array([[ 1,  0,  0,  0],
       [ 0, -1,  0,  0],
       [ 0,  0, -1,  0],
       [ 0,  0,  0,  1]])

In [26]:
#H = XZ + ZI*2 # -ZX + 2*ZI

H = XX*3 + ZZ*7

min(scipy.linalg.eig(H.get())[0])

(-10+0j)

In [27]:
def ansatz(reg, params):
    n_qubits = len(reg)
    depth = n_qubits
    for i in range(depth):
        for j in range(n_qubits):
            if(j < n_qubits-1):
                reg[j+1].CNOT(reg[j])
            reg[i].RY(params[j])
            
def ansatz_2q(q1, q2, params):
    q2.CNOT(q1)
    q1.RY(params[0])
    q2.RY(params[1])
    q1.CNOT(q2)
    q1.RY(params[2])
    q2.RY(params[3])
    q2.CNOT(q1)
    q1.RY(params[4])
    q2.RY(params[5])
    q1.CNOT(q2)

In [28]:
def expectation_2q(params):
    logicQuBit  = LogicQuBit(2)
    q1 = Qubit()
    q2 = Qubit()

    ansatz_2q(q1,q2,params)
    psi = logicQuBit.getPsi()
    
    return (psi.adjoint()*H*psi).get()[0][0]

minimum = minimize(expectation_2q, [0,0,0,0,0,0], method='Nelder-Mead', options={'xtol': 1e-10, 'ftol': 1e-10})
print(minimum)

 final_simplex: (array([[ 5.31847022, -0.42327127, -3.1307354 ,  1.06588762,  1.01423211,
        -3.42077752],
       [ 5.31847022, -0.42327127, -3.1307354 ,  1.06588762,  1.01423211,
        -3.42077752],
       [ 5.31847022, -0.42327127, -3.1307354 ,  1.06588762,  1.01423211,
        -3.42077752],
       [ 5.31847022, -0.42327127, -3.1307354 ,  1.06588762,  1.01423211,
        -3.42077752],
       [ 5.31847022, -0.42327127, -3.1307354 ,  1.06588762,  1.01423211,
        -3.42077752],
       [ 5.31847022, -0.42327127, -3.1307354 ,  1.06588762,  1.01423211,
        -3.42077752],
       [ 5.31847022, -0.42327127, -3.1307354 ,  1.06588762,  1.01423211,
        -3.42077752]]), array([-10., -10., -10., -10., -10., -10., -10.]))
           fun: -10.000000000000004
       message: 'Optimization terminated successfully.'
          nfev: 1065
           nit: 614
        status: 0
       success: True
             x: array([ 5.31847022, -0.42327127, -3.1307354 ,  1.06588762,  1.01423211,
     

In [29]:
def expectation_value(measurements, base = np.array([1,-1,-1,1])):
    probabilities = np.array(measurements)
    expectation = np.sum(base * probabilities)
    return expectation

def sigma_xx(params):
    logicQuBit  = LogicQuBit(2, first_left = False)
    q1 = Qubit()
    q2 = Qubit()
    
    ansatz_2q(q1,q2,params)
    
    # medidas em XX
    q1.RY(-pi/2)
    q2.RY(-pi/2)
    
    result = logicQuBit.Measure([q1,q2])
    result = expectation_value(result)
    return result

def sigma_yy(params):
    logicQuBit  = LogicQuBit(2, first_left = False)
    q1 = Qubit()
    q2 = Qubit()
    
    ansatz_2q(q1,q2,params)
    
    # medidas em YY
    q1.RX(pi/2)
    q2.RX(pi/2)
    
    result = logicQuBit.Measure([q1,q2])
    yy = expectation_value(result)
    iy = expectation_value(result, sig_is)
    yi = expectation_value(result, sig_si)
    return yy, iy, yi

def sigma_zz(params):
    logicQuBit  = LogicQuBit(2, first_left = False)
    q1 = Qubit()
    q2 = Qubit()
    
    ansatz_2q(q1,q2,params)
          
    result = logicQuBit.Measure([q1,q2])
    zz = expectation_value(result)
    iz = expectation_value(result, sig_is) # [zz, iz] = 0
    zi = expectation_value(result, sig_si) # [zz, zi] = 0
    return zz, iz, zi

def sigma_zx(params):
    logicQuBit  = LogicQuBit(2, first_left = False)
    q1 = Qubit()
    q2 = Qubit()
    
    ansatz_2q(q1,q2,params)
    
    # medidas em X
    #q1.RY(-pi/2)
    q2.RY(-pi/2)
    
    result = logicQuBit.Measure([q1,q2])
    zx = expectation_value(result)
    return zx

def sigma_xx_zz(params):
    logicQuBit  = LogicQuBit(2)
    q1 = Qubit()
    q2 = Qubit()
    
    ansatz_2q(q1,q2,params)
    
    # medida na base de bell
    q2.CX(q1)
    q1.H()    # Hx1
    
    result = logicQuBit.Measure([q1,q2])
    zi = expectation_value(result, sig_si)
    iz = expectation_value(result, sig_is)
    xx = zi
    zz = iz
    return xx, zz

def expectation_energy(params):
    #xx =  sigma_xx(params)
    #yy =  sigma_yy(params)
    
    #zz, iz, zi = sigma_zz(params)
    #zx = sigma_zx(params)   
    #result = -zx + zi*2
    
    xx, zz = sigma_xx_zz(params) 
    result = xx*3 + zz*7
    return result

In [30]:
minimum = minimize(expectation_energy, [0,0,0,0,0,0], method='Nelder-Mead', options={'xtol': 1e-10, 'ftol': 1e-10})
print(minimum)

 final_simplex: (array([[ 2.19396352, -0.53207923,  0.91245827, -3.03402174,  0.31749551,
        -1.89342844],
       [ 2.19396352, -0.53207923,  0.91245827, -3.03402174,  0.31749551,
        -1.89342844],
       [ 2.19396352, -0.53207923,  0.91245827, -3.03402174,  0.31749551,
        -1.89342844],
       [ 2.19396352, -0.53207923,  0.91245827, -3.03402174,  0.31749551,
        -1.89342844],
       [ 2.19396352, -0.53207923,  0.91245828, -3.03402174,  0.31749551,
        -1.89342844],
       [ 2.19396352, -0.53207923,  0.91245827, -3.03402174,  0.31749551,
        -1.89342844],
       [ 2.19396352, -0.53207923,  0.91245827, -3.03402174,  0.31749551,
        -1.89342844]]), array([-10., -10., -10., -10., -10., -10., -10.]))
           fun: -10.000000000000005
       message: 'Optimization terminated successfully.'
          nfev: 932
           nit: 543
        status: 0
       success: True
             x: array([ 2.19396352, -0.53207923,  0.91245827, -3.03402174,  0.31749551,
      

In [31]:
def gradient(params, evaluate, shift = pi/2):
    n_params = params.shape[0]
    gradients = np.zeros(n_params)
    
    for i in range(n_params):
        #parameter shift rule
        shift_vect = np.array([shift if j==i else 0 for j in range(n_params)])
        shift_right = params + shift_vect
        shift_left = params - shift_vect
        
        expectation_right = evaluate(shift_right)
        expectation_left = evaluate(shift_left)

        gradients[i] = expectation_right - expectation_left

    return gradients

In [36]:
params = np.random.uniform(-np.pi, np.pi, 6)
last_params = np.zeros(6)

In [37]:
lr = 0.1
err = 1
while err > 1e-3:
    grad = gradient(params, expectation_energy, pi/1.3)
    params = params - lr*grad
    err = abs(sum(params - last_params))
    last_params = np.array(params)
    print(err) 

/tmp/ipykernel_10773/2399136664.py:14: ComplexWarning: Casting complex values to real discards the imaginary part
  gradients[i] = expectation_right - expectation_left


5.242381289750311
1.2889758380243934
0.8469000184874783
1.715505786108866
1.2857087151314395
0.6665949071449118
0.6217152863618391
0.4502480346945008
0.4015030288008472
0.3084211700757258
0.26803408924379646
0.21245510675985635
0.18154598875298378
0.14636982828737843
0.1237463383802897
0.10074594911990808
0.08459198537134305
0.06927493025400655
0.05790614903768021
0.04759712440969022
0.03966686651250573
0.03268382882723664
0.027183388111598594
0.022434057092212578
0.018633066423850764
0.015394340051459787
0.01277418938381536
0.010561655143450865
0.00875847779291572
0.007245152363900109
0.006005594294535843
0.004969651583073942
0.004118184616198395
0.0034086270007260833
0.0028240437050082168
0.0023378482643301446
0.0019366364025408256
0.001603400193223914
0.0013281053403841891
0.0010996639112277329
0.0009107986294276671


In [38]:
expectation_energy(params)

(-9.999999421740936+0j)